## 2021-12-09 Attach mag vector metadata to joint luminosity function fits files ##

**2021-12-09** Started (Will Clarkson). The working directory currently 
contains file jointlf_gi.fits .

In [1]:
from astropy.table import Table
import numpy as np
import os

In [2]:
def parseFilter(filts='zz', lFilts=[]):
    
    """Returns matching joint luminosity function file to two-filter input string. 
    
    Example call:
    filLF = parseFilter('gi')
    
    
    Arguments:
    
    filts = two-letter string for the filters of interest (e.g. 'gi', 'ur', ...)
    
    lFilts = user-supplied list of joint-LF filenames. Defaults to canned list if not set
    
    If no match found, returns a blank string. Otherwise, returns the matching element in
    the list of LF files. Matches are order-insensitive, so filts='gi' and 'ig' would both
    match the file 'jointlf_gi.fits'  
    """
    
    ## Because the way the noirlab datalab paths work is not entirely clear to me,
    ## I set up a default list in this method.
    lDefault = ['jointlf_gi.fits','jointlf_gr.fits', \
                'jointlf_gz.fits','jointlf_iz.fits,', \
                'jointlf_ri.fits','jointlf_rz.fits,', \
                'jointlf_ug.fits','jointlf_ui.fits',\
                'jointlf_ur.fits','jointlf_uz.fits']
    if len(lFilts) < 1:
        lFilts = lDefault[:]
        
    filMat=''
    for filCand in lFilts:
        sPair = os.path.splitext(filCand)[0].split('_')[-1]
        if sPair.find(filts.lower()) == 0 or sPair.find(filts[::-1].lower()) == 0:
            filMat = filCand[:]
            break

    return filMat

In [3]:
def buildLinspacePars():
    
    """Utility to return linspace parameters for metadata. Example call:
    
    dparams = buildLinspacePars()"""
    
    # This is just to record the parameters
    
    Dranges={}
    Dranges['gi'] = []
    Dranges['gi'].append({'start':15., 'stop':30., 'num':75, 'filtname':'g'})
    Dranges['gi'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'i'})
    
    Dranges['gr'] = []
    Dranges['gr'].append({'start':15., 'stop':30., 'num':75, 'filtname':'g'})
    Dranges['gr'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'r'})

    Dranges['gz'] = []
    Dranges['gz'].append({'start':15., 'stop':30., 'num':75, 'filtname':'g'})
    Dranges['gz'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'z'})

    Dranges['iz'] = []
    Dranges['iz'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'i'})
    Dranges['iz'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'z'})

    Dranges['ri'] = []
    Dranges['ri'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'r'})
    Dranges['ri'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'i'})

    Dranges['rz'] = []
    Dranges['rz'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'r'})
    Dranges['rz'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'z'})

    Dranges['ug'] = []
    Dranges['ug'].append({'start':15., 'stop':33., 'num':90, 'filtname':'u'})
    Dranges['ug'].append({'start':15, 'stop':40., 'num':75, 'filtname':'g'})

    Dranges['ui'] = []
    Dranges['ui'].append({'start':15., 'stop':33., 'num':90, 'filtname':'u'})
    Dranges['ui'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'i'})

    Dranges['ur'] = []
    Dranges['ur'].append({'start':15., 'stop':33., 'num':90, 'filtname':'u'})
    Dranges['ur'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'r'})

    Dranges['uz'] = []
    Dranges['uz'].append({'start':15., 'stop':33., 'num':90, 'filtname':'u'})
    Dranges['uz'].append({'start':12.5, 'stop':27.5, 'num':75, 'filtname':'z'})

    return Dranges
    

In [4]:
def attachLinspace(filLF='jointlf_gi.fits', suffix='_wVecs', dpars={}, \
                   Verbose=False):
    
    """Attaches linspace info to table metadata. Example call:
    
    attachLinspace('jointLF_gi.fits')
    
    ... which will, if successful, populate file jointLF_gi_imported_wVecs.fits
    
    Returns True if successful.
    
    Arguments:
    
    filLF = path to joint-LF file
    
    suffix = suffix to add for output filename.
    
    dpars = dictionary of linspace parameters for each filter. Will be generated
    as default if not supplied. (Default: leave this argument empty).
    
    Verbose = print screen output when reading and writing the joint-LF files
    
    """
    
    # This uses the file to look up the linspace parameters for attachment
    
    # get the filter-pair key
    spair = filLF.split('_')[1].split('.')[0]

    # get the linspace info if not supplied (as is the default)
    if len(dpars.keys()) < 1:
        dpars = buildLinspacePars()
    
    if not spair in dpars.keys():
        print("attachLinspace FATAL - linspace info matching %s not found." % (spair))
        return False
        
    # generate the output filename
    pathParts = os.path.splitext(filLF)
    pathOut = '%s%s%s' % (pathParts[0], suffix, pathParts[-1])
    
    # now actually do the metadata
    try:
        if Verbose:
            print("attachLinspace INFO: Loading path %s..." % (filLF))
        tLF = Table.read(filLF)
    except:
        print("attachLinspace WARN - cannot read path %s" % (filLF))     
        return False
        
    # Re-generate the vec1 and vec2 from the parameters
    pars = dpars[spair]
    mags0 = np.linspace(pars[0]['start'], pars[0]['stop'], pars[0]['num']+1, endpoint=True)
    mags1 = np.linspace(pars[1]['start'], pars[1]['stop'], pars[1]['num']+1, endpoint=True)

    # fits format leads to uppercase keyword names, so let's make these upper-case
    # from the beginning to keep things consistent later on
    tLF.meta['FILT0'] = pars[0]['filtname']
    tLF.meta['FILT1'] = pars[1]['filtname']
    tLF.meta['MAGS0'] = list(mags0)
    tLF.meta['MAGS1'] = list(mags1)
    
    # Now write this to disk
    try:
        if Verbose:
            print("attachLinspace INFO: Writing path %s ..." % (pathOut))
        tLF.write(pathOut, overwrite=True)
        return True
    except:
        print("attachLinspace WARN: problem writing path %s" % (pathOut))
        return False

### Example calls ###

In [5]:
# example parsing user-input filter string to match with luminosity function file
# (allowing for wrong ordering and case in user input)
print(parseFilter('Ig'))

jointlf_gi.fits


In [6]:
# Example call attaching mag vector info to joint-LF file
attachLinspace('jointlf_gi.fits', Verbose=True)

attachLinspace INFO: Loading path jointlf_gi.fits...
attachLinspace INFO: Writing path jointlf_gi_wVecs.fits ...


True

In [7]:
#attachLinspace('jointlf_gi.fits', Verbose=True)
attachLinspace('jointlf_gr.fits', Verbose=True)
attachLinspace('jointlf_gz.fits', Verbose=True)

attachLinspace('jointlf_ug.fits', Verbose=True)
attachLinspace('jointlf_ui.fits', Verbose=True)
attachLinspace('jointlf_ur.fits', Verbose=True)
attachLinspace('jointlf_uz.fits', Verbose=True)

attachLinspace('jointlf_ri.fits', Verbose=True)
attachLinspace('jointlf_rz.fits', Verbose=True)

attachLinspace('jointlf_iz.fits', Verbose=True)



attachLinspace INFO: Loading path jointlf_gr.fits...
attachLinspace INFO: Writing path jointlf_gr_wVecs.fits ...
attachLinspace INFO: Loading path jointlf_gz.fits...
attachLinspace INFO: Writing path jointlf_gz_wVecs.fits ...
attachLinspace INFO: Loading path jointlf_ug.fits...
attachLinspace INFO: Writing path jointlf_ug_wVecs.fits ...
attachLinspace INFO: Loading path jointlf_ui.fits...
attachLinspace INFO: Writing path jointlf_ui_wVecs.fits ...
attachLinspace INFO: Loading path jointlf_ur.fits...
attachLinspace INFO: Writing path jointlf_ur_wVecs.fits ...
attachLinspace INFO: Loading path jointlf_uz.fits...
attachLinspace INFO: Writing path jointlf_uz_wVecs.fits ...
attachLinspace INFO: Loading path jointlf_ri.fits...
attachLinspace INFO: Writing path jointlf_ri_wVecs.fits ...
attachLinspace INFO: Loading path jointlf_rz.fits...
attachLinspace INFO: Writing path jointlf_rz_wVecs.fits ...
attachLinspace INFO: Loading path jointlf_iz.fits...
attachLinspace INFO: Writing path jointlf_i

True

In [8]:
# Example reading the augmented LF file with the magnitude vectors
tTest = Table.read('jointlf_gi_wVecs.fits')
vec0 = np.array(tTest.meta['MAGS0'])
vec1 = np.array(tTest.meta['MAGS1'])
filt0 = tTest.meta['FILT0']
filt1 = tTest.meta['FILT1']

print(filt0, vec0[0:10])
print(filt1, vec1[0:10])

g [15.  15.2 15.4 15.6 15.8 16.  16.2 16.4 16.6 16.8]
i [12.5 12.7 12.9 13.1 13.3 13.5 13.7 13.9 14.1 14.3]


In [10]:
filt0

'g'